In [ ]:
import numpy as np 
from sklearn import datasets
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import DecisionBoundaryDisplay

In [ ]:
def sigmoid(array: np.ndarray) -> np.ndarray:
    sign = np.sign(array)
    exp = np.exp(-sign * array)

    num = np.where(sign < 0, exp, 1)
    den = 1 + exp

    return num / den

In [ ]:
def log1pexp(array: np.ndarray) -> np.ndarray:
    thresh_1 = -37
    thresh_2 = 18
    thresh_3 = 34

    lower_than_1 = (array <= thresh_1).astype(int)
    lower_than_2 = (array <= thresh_2).astype(int)
    lower_than_3 = (array <= thresh_3).astype(int)
    lower = lower_than_1 + lower_than_2 + lower_than_3

    res = np.empty_like(array)
    res[lower==0] = array[lower==0]
    res[lower==1] = array[lower==1] + np.exp(-array[lower==1])
    res[lower==2] = np.log1p(np.exp(array[lower==2]))
    res[lower==3] = np.exp(array[lower==3])
    
    return res

In [ ]:
iris = datasets.load_iris()
samples = iris.data[:, :2]  # we only take the first two features.
gt_classes = iris.target

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
for label in np.unique(gt_classes):
    ax.scatter(
        samples[gt_classes==label, 0], 
        samples[gt_classes==label, 1], 
        label=iris.target_names[label]
    )
ax.legend()

In [ ]:
logreg = LogisticRegression(solver="newton-cg")
logreg.fit(samples, (gt_classes.copy() == 0).astype(int))

fig, ax = plt.subplots(1, 1, figsize=(7, 5))
DecisionBoundaryDisplay.from_estimator(
    logreg,
    samples,
    cmap=plt.cm.Paired,
    ax=ax,
    response_method="predict",
    plot_method="pcolormesh",
    shading="auto",
    xlabel="Sepal length",
    ylabel="Sepal width",
    eps=0.5,
)

for label in np.unique(gt_classes):
    ax.scatter(
        samples[gt_classes==label, 0], 
        samples[gt_classes==label, 1], 
        label=iris.target_names[label]
    )
ax.legend()

### Maximum Likelihood

In [ ]:
nsamples = samples.shape[0]
X = np.concat([samples.T, np.ones((1, nsamples))], axis=0)
W = gt_classes.copy() == 0
W = W.astype(int)
one_minus_W = 1 - W
XXT = np.einsum("ij, kj -> ikj", X, X)

In [ ]:
def objective(phi: np.ndarray) -> np.ndarray:
    phi_X = np.dot(phi, X)

    sign = np.sign(phi_X)
    exp = np.exp(-sign * phi_X)
    log1pexp = np.log1p(exp)

    obj = one_minus_W * phi_X
    obj += np.where(sign >= 0, log1pexp, log1pexp - phi_X)

    return np.sum(obj) / nsamples

def objective_jac(phi: np.ndarray) -> np.ndarray:
    sig_minus_W = sigmoid(np.dot(phi, X)) - W
    return np.einsum("j, ij -> i", sig_minus_W, X) / nsamples

def objective_hess(phi: np.ndarray) -> np.ndarray:
    sig_phi_X = sigmoid(np.dot(phi, X))
    return np.einsum("k, ijk -> ij", sig_phi_X * (1 - sig_phi_X), XXT) / nsamples

In [ ]:
opt_results = minimize(
    fun=objective,
    x0=np.full((X.shape[0],), 0),
    method="Newton-CG",
    jac=objective_jac,
    hess=objective_hess,
    options={"maxiter": 100},
)
print(opt_results)

phi_opt = opt_results.x

In [ ]:
xrange = np.array([0, 8])
yrange = - (phi_opt[0] * xrange + phi_opt[2]) / phi_opt[1]

In [ ]:
grid_x, grid_y = np.meshgrid(
    np.linspace(xrange[0], xrange[1], 1000),
    np.linspace(yrange[0], yrange[1], 1000),
    indexing="xy"
)
grid_coords = np.stack([grid_x, grid_y, np.ones_like(grid_x)], axis=-1)
activations = grid_coords @ phi_opt
likelihood_full = sigmoid(activations)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
ax.imshow(
    likelihood_full,
    cmap="hot",
    extent=[xrange[0], xrange[1], yrange[0], yrange[1]],
    origin="lower",
    aspect="auto",
)
for label in np.unique(gt_classes):
    ax.scatter(
        samples[gt_classes==label, 0], 
        samples[gt_classes==label, 1], 
        label=iris.target_names[label]
    )
ax.plot(xrange, yrange, "g")
ax.set_xlim(samples[:, 0].min()-0.1, samples[:, 0].max()+0.1)
ax.set_ylim(samples[:, 1].min()-0.1, samples[:, 1].max()+0.1)
ax.legend()

### Bayesian Classification

In [ ]:
X = np.concatenate([samples.T, np.ones((1, samples.shape[0]))], axis=0)
W = gt_classes.copy() == 0
W = W.astype(int)
one_minus_W = 1 - W

nsamples = X.shape[1]
sigma_p = 1000

XXT = np.einsum("kj, ij -> kij", X, X)
for i in range(nsamples):
    assert np.allclose(X[:,i:i+1] @ X[:, i:i+1].T, XXT[..., i])
    assert np.allclose(np.einsum("i, j -> ij", X[:,i], X[:, i]), XXT[..., i])

In [ ]:
def objective(phi: np.ndarray) -> np.ndarray:
    obj_1 = 1.0 / (2 * sigma_p) * phi @ phi

    activations = np.einsum("ij, i -> j", X, phi)
    obj_2 = np.logaddexp(0, -activations) + one_minus_W * activations

    return obj_1 + obj_2.sum()

def objective_jac(phi: np.ndarray) -> np.ndarray:
    obj_1 = 1.0 / sigma_p * phi
    
    activations = np.einsum("ij, i -> j", X, phi)
    obj_2 = (sigmoid(activations) - W) * X 

    return obj_1 + np.sum(obj_2, axis=1)

def objective_hess(phi: np.ndarray) -> np.ndarray:
    obj_1 = 1 / sigma_p * np.eye(phi.shape[0])

    activations = np.einsum("ij, i -> j", X, phi)
    proba = sigmoid(activations)
    obj_2 = proba * (1 - proba) * XXT

    return obj_1 + np.sum(obj_2, axis=2)

In [ ]:
laplace_mean = minimize(
    objective,
    np.zeros((X.shape[0],)),
    method="Newton-CG",
    jac=objective_jac,
    hess=objective_hess,
    options={"maxiter": 100},
).x

laplace_cov = objective_hess(laplace_mean)

In [ ]:
xrange = np.array([4, 8])
yrange = np.array([1.9, 4.5])
grid_x, grid_y = np.meshgrid(
    np.linspace(xrange[0], xrange[1], 100),
    np.linspace(yrange[0], yrange[1], 100),
    indexing="xy"
)
grid_coords = np.stack([grid_x, grid_y, np.ones_like(grid_x)], axis=-1)

In [ ]:
laplace_means = np.einsum("ijk, k -> ij", grid_coords, laplace_mean)
laplace_covs = np.einsum("ijk, kl, ijl -> ij", grid_coords, laplace_cov, grid_coords)
laplace_log_factors = -0.5 * (np.log(2 * np.pi) + np.log(laplace_covs))

In [ ]:
def log_gaussian(activation: float, i: int, j: int) -> np.ndarray:
    diff = (activation - laplace_means[i, j]) ** 2
    
    return laplace_log_factors[i, j] - 0.5 * diff / laplace_covs[i, j]

def gaussian(activation: float, i: int, j: int) -> np.ndarray:
    return np.exp(log_gaussian(activation, i, j))

def bernouilli(activation: float) -> float:
    return sigmoid(activation)

def joint_proba(activation: float, i: int, j: int) -> np.ndarray:
    return bernouilli(activation) * gaussian(activation, i, j)

In [ ]:
import scipy.integrate as integrate
from itertools import product

likelihood_full = np.empty_like(grid_x)
for i, j in product(np.arange(grid_x.shape[0]), np.arange(grid_x.shape[1])):
    likelihood_full[i, j] = integrate.quad(lambda x: joint_proba(x, i, j), -np.inf, np.inf)[0]

In [ ]:
log_likelihood_approx = -np.logaddexp(
    0, 
    -laplace_means / np.sqrt(1 + np.pi * laplace_covs / 8)
)
likelihood_approx = np.exp(log_likelihood_approx)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
ax.imshow(
    likelihood_full,
    cmap="hot",
    extent=[xrange[0], xrange[1], yrange[0], yrange[1]],
    origin="lower",
    aspect="auto",
)
ax.contour(grid_x, grid_y, likelihood_full, levels=[0.5])
for label in np.unique(gt_classes):
    ax.scatter(
        samples[gt_classes==label, 0], 
        samples[gt_classes==label, 1], 
        label=iris.target_names[label]
    )
ax.set_xlim(samples[:, 0].min()-0.1, samples[:, 0].max()+0.1)
ax.set_ylim(samples[:, 1].min()-0.1, samples[:, 1].max()+0.1)
ax.legend()